In [1]:
from os import path
import string
import sys
sys.path.append("..")

import pandas as pd

from src.Dataset import Dataset
from src.visualisations import *
from src.data_processing import *
from src.model import *
import re

In [2]:
train_set = path.join('..', 'data', 'training_set.json')
test_set = path.join('..', 'data', 'testing_set.json')
data = Dataset(train_set, test_set)

Loaded 7100 rows


# Création d'un nouveau modèle
## Traitement des données
### Traitement du déséquilibre interclasses
Avant toute chose, il est nécessaire de résoudre le problème du déséquilibre entre le nombre de phrases dans chaque classe.
Pour cela, nous avons décidé de simplement retirer un certain nombre de phrases (aléatoirement) de la classe "irrelevant" afin que toutes les classes aient en moyenne à peu près le même nombre de phrases que les autres classes

Peut-être :
- Tout mettre au meme niveau
- Data augmentation

In [3]:
balanced_data = balance(data)

### Préparation pour l'entrainement
#### Vectorisation
On vectorise tout

In [4]:
vectorized_sentences = vectorize_data(balanced_data['sentence'])

#### One hot encodage

In [5]:
one_hot_labels = to_one_hot(balanced_data['intent'])

## Le modèle

In [6]:
model = Model(vectorized_sentences.shape[1])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               505900    
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 408       
Total params: 511,358
Trainable params: 511,358
Non-trainable params: 0
_________________________________________________________________
None


On redécoupe les données en en gardant 20% pour tester le modèle

In [7]:
vectorized_sentences.shape

(2883, 5058)

In [8]:
one_hot_labels.shape

(2883, 8)

In [9]:
sentences_train, sentences_test, y_train, y_test = cut_data(vectorized_sentences, one_hot_labels, 0.2)

On entraine le modèle

In [10]:
model.train(sentences_train, y_train, sentences_test, y_test, True)

Epoch 1/100
231/231 [==============================] - 0s 2ms/step - loss: 1.4552 - accuracy: 0.5026 - val_loss: 0.7707 - val_accuracy: 0.8007
Epoch 2/100
231/231 [==============================] - 0s 1ms/step - loss: 0.3483 - accuracy: 0.9267 - val_loss: 0.4874 - val_accuracy: 0.8614
Epoch 3/100
231/231 [==============================] - 0s 1ms/step - loss: 0.0742 - accuracy: 0.9905 - val_loss: 0.4714 - val_accuracy: 0.8579
Epoch 4/100
231/231 [==============================] - 0s 1ms/step - loss: 0.0215 - accuracy: 0.9987 - val_loss: 0.4894 - val_accuracy: 0.8527
Epoch 5/100
231/231 [==============================] - 0s 1ms/step - loss: 0.0097 - accuracy: 0.9996 - val_loss: 0.5084 - val_accuracy: 0.8544
Epoch 6/100
231/231 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.5287 - val_accuracy: 0.8579
Epoch 7/100
231/231 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.5537 - val_accuracy: 0.8579

Epoch 57/100
231/231 [==============================] - 0s 1ms/step - loss: 8.4940e-07 - accuracy: 1.0000 - val_loss: 1.1170 - val_accuracy: 0.8440
Epoch 58/100
231/231 [==============================] - 0s 1ms/step - loss: 7.4374e-07 - accuracy: 1.0000 - val_loss: 1.1281 - val_accuracy: 0.8458
Epoch 59/100
231/231 [==============================] - 0s 1ms/step - loss: 6.4965e-07 - accuracy: 1.0000 - val_loss: 1.1370 - val_accuracy: 0.8458
Epoch 60/100
231/231 [==============================] - 0s 1ms/step - loss: 5.7185e-07 - accuracy: 1.0000 - val_loss: 1.1457 - val_accuracy: 0.8423
Epoch 61/100
231/231 [==============================] - 0s 1ms/step - loss: 5.0429e-07 - accuracy: 1.0000 - val_loss: 1.1557 - val_accuracy: 0.8440
Epoch 62/100
231/231 [==============================] - 0s 1ms/step - loss: 4.4515e-07 - accuracy: 1.0000 - val_loss: 1.1663 - val_accuracy: 0.8440
Epoch 63/100
231/231 [==============================] - 0s 1ms/step - loss: 3.9128e-07 - accuracy: 1.0000 - val_

## Evaluations
Evaluons le modèle pour le comparer avec le modèle que nous avions précédemment

## Sauvegarde
Pour finir, on sauvegarde le modèle afin de l'utiliser dans une API

In [12]:
model.save_model('simple_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../models/simple_model/assets
